In [ ]:
from pycorenlp import StanfordCoreNLP
from nltk import tree
import re

nlp = StanfordCoreNLP('http://localhost:9000')

with open('kwords.json', 'r') as f:
    s = ' '.join([l for l in f])
    kw = eval(s)
params_kw = {kw_['name']:kw_['keywords'] for kw_ in kw if kw_['group'] == 'parameter'}

In [ ]:
filter_kw = {kw_['name']:{'keywords':kw_['keywords'],
                         'type':kw_['type'],
                         'categories':kw_['categories']} 
                         if 'categories' in kw_ else
                         {'keywords':kw_['keywords'],
                                     'type':kw_['type'],
                                     'condition_kw':kw_['condition_kw']} for kw_ in kw if kw_['group'] == 'parameter'}

In [ ]:
def match_param(w):
    for p_name, p_vals in params_kw.iteritems():
        for p_val in p_vals:
            if re.match(p_val,w):
                return p_name
    return 0

In [ ]:
def match_filter(w):
    result = []
    for p_name, p_vals in filter_kw.iteritems():
        
        for p_val in p_vals['keywords']:
            if re.findall(p_val,w):
                result.append(p_name)
                break
    return result

In [ ]:
def find_group(itree,pos=[]):
    if not isinstance(itree, unicode):
        if itree.label() == 'IN':
            if itree[0].lower() == 'by':
                return pos
        else:
            i = 0
            for subtree in itree:
                res = find_group(subtree,pos+[i])
                i += 1
                if not res is None:
                    return res
    return None 

In [ ]:
## -- Remove or improve -- ##
def get_first_nn(itree):
    if not isinstance(itree, unicode):
        if itree.label() == 'NP':
            if len(itree) == 1 and itree[0].label() == 'PRP':
                return 0
            for subtree in itree:
                res = get_first_nn(subtree)
                if res:
                    return res
            return itree
        else:
            for subtree in itree:
                res = get_first_np(subtree)
                if res:
                    return res
    return 0

In [ ]:
def get_first_np(itree):
    if not isinstance(itree, unicode):
        if itree.label() == 'NP':
            if len(itree) == 1 and itree[0].label() == 'PRP':
                return 0
            for subtree in itree:
                res = get_first_np(subtree)
                if res:
                    return res
            return itree
        else:
            for subtree in itree:
                res = get_first_np(subtree)
                if res:
                    return res
    return 0

In [ ]:
def get_all_nps(itree, l=[]):
    if not isinstance(itree, unicode):
        if itree.label() == 'NP':
            if len(itree) == 1 and itree[0].label() == 'PRP':
                return l
            l2 = len(l)
            for subtree in itree:
                l = get_all_nps(subtree, l)
            if l2 == len(l):
                l.append(itree)
        else:
            if itree.label() == 'PP' and itree.height() <= 3:
                l.append(itree)
            else:
                for subtree in itree:
                    l = get_all_nps(subtree, l)
    return l

In [ ]:
def preproc(istr):
    ostr = re.sub(r'((?=\D) %|% (?=\D))', ' percentage ', istr)
    #ostr = re.sub(r'(?i)\b(I|me|we|us|you)\b', '', istr)
    return ostr

In [ ]:
mytree = tree.Tree.fromstring(nlp.annotate('Show me distribution by age among males in Texas', 
             properties={
                'annotators': 'parse',
                'outputFormat': 'json'
                })['sentences'][0]['parse'])

In [ ]:
def parse_it(text):
    text2 = preproc(text)
    
    output = nlp.annotate(text2, properties={
      'annotators': 'parse',
      'outputFormat': 'json'
      })
    for oresult in output['sentences']:
        result = oresult['parse']
        mytree = tree.Tree.fromstring(result)
        parameters = {}
        target = ''
        
        if mytree[0].label()=='SBARQ':
            if mytree[0][0].label()=='WHNP':
                if mytree[0][0][0].label()=='WP':
                    if mytree[0][0][0][0].lower() == 'who':
                        parameters['mode'] = 'model'
                        print 'Who'
                        print('-- run np search for target --')
                    else:
                        print 'What1'
                        print('-- run np search for target --')
                    
                    # question body #
                elif mytree[0][0][0].label()=='WHNP' and mytree[0][0][0][0].label()=='WHADJP':
                    parameters['mode'] = 'query'
                    parameters['aggregation'] = 'count'
                    st = mytree[0][0][0].flatten()
                    for i in range(len(st)):
                        target = match_param(st[-i-1])
                        if target:
                            break
                    print('-- run np search for target --')

                elif mytree[0][0][0].label()=='WDT':
                    parameters['mode'] = 'query'
                    print 'What2'
                    print('-- run np search for target --')
                    
                    # search target in question question #
                else:
                    print('-- run np search for target -- --- no question type ---')
            else:
                print '__'
        
        elif (mytree[0].label()=='S' and mytree[0][0].label()=='VP') or mytree[0].label()=='SQ':
            if mytree[0].label()!='SQ':
                if mytree[0][0][0].label() == 'VB' and mytree[0][0][0] in ['list','table']:
                    parameters['view'] = ['table']
            print('-- run np search for target -- --- no question type ---')
            
            # check for internal sentence
        elif mytree[0].label()=='NP':
            print('-- run np search for target -- --- no question type ---')
            
            # search target #
        else:
            print mytree
            
            # search target #
        
        if not target:
            st = get_first_np(mytree).flatten()
            for i in range(len(st)):
                target = match_param(st[-i-1])
                if target:
                    break
                    
        #nps = get_all_nps(mytree, [])
        #nn = get_first_np(mytree)
        
        group_by = []
        address = find_group(mytree)
        if address:
            group_st = mytree[address[:-1]]
            group_st = get_first_np(group_st)
            group_st = ' '.join(group_st.flatten())
            group_by = match_filter(group_st)
        
        s = ' '.join(mytree.flatten())
        filter_by = match_filter(s)
        
        filter_by = [tmp for tmp in filter_by if tmp not in group_by]
        filter_by = [tmp for tmp in filter_by if tmp not in [target, 'customer']]
        
        print 'tree:'
        print mytree.flatten()
        
        print 'target:'
        print target
        
        print 'filter:'
        print filter_by
        
        print 'group_by'
        print group_by
        
        print '\n\n'

In [ ]:
parse_it(
    "How customers who use credit cards differ from others? "

    "What is average age of customers by state? "
    'Show us total age of those customers who has credit cards. '
    "Do you know what is average customer age? "
    'How many males vs females % ? '
    'Who is my target for selling debt funds? '
    'Males by state. '
    'Average income of our customers by age. '
    
    """
    How many males vs females %?
    What % have insurance?
    Who is my target for selling debt funds?
    Who is my target for selling retirement products?
    Show me who my target for selling debt funds is.
    Show me age distribution of those customers who have credit card.

    Show me gender distribution by states.
    Show me gender distribution by age.
    Show me gender distribution by income.
    Show me gender distribution by employment status.
    What is gender distribution of my customers?
    What is gender distribution of customers with insurance?

    Show me distribution by age among males.
    Show me distribution by age among males in Texas.

    What is average income by age?

    How many customers in Alabama has more than one child?
    How many customers in Alabama has more than two children?
    How many customers in Alabama has no children?

    Show me average number of children by state.

    Show me average income by gender.

    What average income is by employment status?

    Show me 10 customers who buys most debt funds in New York in this year?

    List customers from Alabama with largest income.

    Who should I target for selling credit cards in Florida?

    Who usually buys debit cards?
    Who usually buys debit cards among unemployed people?

    Show me employment status by gender.
    Show me employment status by age for women.

    How many percentage has family?
    Show me how many % has family by age?

    Show me how many customers have no insurance by state.""")

## Question taxonomy
Our analyses can parse 6 main types of question:
1. **SBARQ**
> Direct question introduced by a wh-word or a wh-phrase. Indirect questions and relative clauses should be bracketed as SBAR, not SBARQ.

    First subtree structures:
    1. __WHNP - WP__  
        Noun questions: (Who?, What?)  
        _Who_ questions are categorized as modelling commands and data is being processed according to model specific scenario.  
        _What_ questions will be processed further  
        <br>
    2. __WHNP - WHNP - WHADJP__  
        _How many_ type of question  
        <br>
    3. __WHNP - WDT__  
        _What_ type of question          
        <br>

2. **S-VP** or **SQ**
    - Simple declarative clause, i.e. one that is not introduced by a (possible empty) subordinating conjunction or a wh-word and that does not exhibit subject-verb inversion.
    - VP - Vereb Phrase.  
    - Inverted yes/no question, or main clause of a wh-question, following the wh-phrase in SBARQ.
    Direct commands, like: _show, display, list_
3. **NP**
    - Noun pharase  
    Simple statement

4. **others**
    For all other forms there is a set of parsing rules

_http://web.mit.edu/6.863/www/PennTreebankTags.html_

In [ ]:
isinstance(mytree[pos],unicode)